In [5]:
import os
os.environ['PYTHONHASHSEED']=str(0)
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = ""
import random
random.seed(0)
import numpy as np
np.random.seed(0)
import tensorflow as tf
tf.compat.v1.set_random_seed(0)
import matplotlib.pyplot as plt
from keras import backend as K
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
import keras
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.applications import ResNet50
from keras.utils import to_categorical
from keras.datasets import mnist
import csv
MNIST_image_path = 'MNIST_data'

Using TensorFlow backend.


In [6]:
import optuna
from optuna.integration import KerasPruningCallback
tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(tf.config.experimental.list_logical_devices('GPU')))

Num GPUs Available:  1


In [7]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

num_classes = 10
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

training_images = x_train
training_labels = y_train
test_images = x_test
test_labels = y_test

del x_train
del x_test
del y_train
del y_test

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [8]:
def create_model(trial):
    
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(10, activation='softmax'))
    
    opt = Adam()
    
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer = opt,
              metrics=['accuracy'])

    model.summary()

    return model

In [5]:
def create_model1(trial):
    
    model = Sequential()
    
    filters1 = trial.suggest_int("filters1", 25, 45)
    #filters1 = int(filters1*50)
    
    model.add(Conv2D(filters1, kernel_size= (3, 3),
            activation='relu', 
            input_shape=input_shape))   
    filters2 = trial.suggest_int("filters2", 45, 75)
    #filters2 = int(filters2*100)
    
    model.add(Conv2D(filters2, kernel_size= (3, 3),
            activation='relu'))                  
    model.add(MaxPooling2D(pool_size=(2,2))) 
    dropout1 = trial.suggest_uniform("dropout1", 0.2, 0.3)
    model.add(Dropout(rate=dropout1))
    
    model.add(Flatten())
    
    num_hidden = trial.suggest_int("n_units", 80, 200)
    #num_hidden = int(num_hidden*150)
    model.add(Dense(num_hidden, activation="relu"))
    
    dropout2 = trial.suggest_uniform("dropout2", 0.4, 0.5)
    model.add(Dropout(rate=dropout2))
       
    model.add(Dense(10, activation="softmax"))
    
    lr = trial.suggest_loguniform("adam_lr", 1e-5, 1e-2)
    opt = Adam(learning_rate = lr)
    
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer = opt,
              metrics=['accuracy'])
    
    model.summary()

    return model

In [6]:
def objective(trial):
    
    # clear clutter from previous session graphs.
    tf.keras.backend.clear_session()

    batch_size = trial.suggest_int("batch_size", 10, 150)
    epochs = 12
    print('Not using data augmentation.')
    
    # Generate our trial model.
    model = create_model1(trial)
    
    # Fit the model on the training data.
    # The KerasPruningCallback checks for pruning condition every epoch.
    history = model.fit(training_images, training_labels, 
              callbacks=[KerasPruningCallback(trial, "val_accuracy")],
              epochs = epochs, batch_size = batch_size,
              validation_data = (test_images, test_labels) , verbose = 2, shuffle = False)
    

    # Evaluate the model accuracy on the test set.
    score = model.evaluate(test_images, test_labels, verbose=0)
    print ('Test loss: {}, Test accuracy: {}'.format(score[0], score[1]*100.0))
    return score[1]

In [7]:
sampler = optuna.samplers.TPESampler()
#sampler = optuna.integration.CmaEsSampler()
study = optuna.create_study(sampler = sampler, direction="maximize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=50)
pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


Not using data augmentation.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 36)        360       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 75)        24375     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 75)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 75)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 10800)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 195)               2106195   
_________________________________________________________________
dropout_2 (Dropout)      

[I 2020-05-23 09:28:23,554] Finished trial#0 resulted in value: 0.9896000027656555. Current best value is 0.9896000027656555 with parameters: {'batch_size': 116, 'filters1': 36, 'filters2': 75, 'dropout1': 0.22804979764363217, 'n_units': 195, 'dropout2': 0.4007854047884365, 'adam_lr': 0.0005677480487539295}.


Not using data augmentation.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 40)        400       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 50)        18050     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 50)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 50)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 153)               1101753   
_________________________________________________________________
dropout_2 (Dropout)      

[I 2020-05-23 09:30:19,982] Finished trial#1 resulted in value: 0.9904999732971191. Current best value is 0.9904999732971191 with parameters: {'batch_size': 40, 'filters1': 40, 'filters2': 50, 'dropout1': 0.21558022806620136, 'n_units': 153, 'dropout2': 0.4110857003627551, 'adam_lr': 0.002593206778261273}.


Not using data augmentation.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 36)        360       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 50)        16250     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 50)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 50)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 107)               770507    
_________________________________________________________________
dropout_2 (Dropout)      

...

Not using data augmentation.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 29)        290       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 47)        12314     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 47)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 47)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6768)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 148)               1001812   
_________________________________________________________________
dropout_2 (Dropout)      

[I 2020-05-23 09:56:22,194] Setting status of trial#48 as TrialState.PRUNED. Trial was pruned at epoch 2.


Not using data augmentation.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 44)        440       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 50)        19850     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 50)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 50)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 7200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 134)               964934    
_________________________________________________________________
dropout_2 (Dropout)      

[I 2020-05-23 09:56:44,306] Setting status of trial#49 as TrialState.PRUNED. Trial was pruned at epoch 1.


Study statistics: 
  Number of finished trials:  50
  Number of pruned trials:  40
  Number of complete trials:  10
Best trial:
  Value:  0.9922000169754028
  Params: 
    batch_size: 56
    filters1: 26
    filters2: 53
    dropout1: 0.26331986854914563
    n_units: 145
    dropout2: 0.42285973964334794
    adam_lr: 0.0011452929824426153


In [10]:
with open("Optuna_result_dict_TPE_MNIST_final50.csv", "w", newline='') as f:
    writer = csv.writer(f)
    writer.writerow(["Results"])
    for trial in study.trials:
        writer.writerow([trial.value])

In [11]:
def objective(trial):
    
    # clear clutter from previous session graphs.
    tf.keras.backend.clear_session()

    batch_size = 128
    epochs = 12
    print('Not using data augmentation.')
    
    # Generate our trial model.
    model = create_model(trial)
    
    # Fit the model on the training data.
    # The KerasPruningCallback checks for pruning condition every epoch.
    history = model.fit(training_images, training_labels, 
              callbacks=[KerasPruningCallback(trial, "val_accuracy")],
              epochs = epochs, batch_size = batch_size,
              validation_data = (test_images, test_labels) , verbose = 2, shuffle = False)
    

    # Evaluate the model accuracy on the test set.
    score = model.evaluate(test_images, test_labels, verbose=0)
    print ('Test loss: {}, Test accuracy: {}'.format(score[0], score[1]*100.0))
    return score[1]

In [12]:
sampler = optuna.samplers.TPESampler()
#sampler = optuna.integration.CmaEsSampler()
study = optuna.create_study(sampler = sampler, direction="maximize", pruner=optuna.pruners.MedianPruner())
study.optimize(objective, n_trials=3)
pruned_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.structs.TrialState.COMPLETE]
print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


Not using data augmentation.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)      

[I 2020-05-25 10:14:41,786] Finished trial#0 resulted in value: 0.9927999973297119. Current best value is 0.9927999973297119 with parameters: {}.


Not using data augmentation.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)      

[I 2020-05-25 10:15:31,710] Finished trial#1 resulted in value: 0.9915000200271606. Current best value is 0.9927999973297119 with parameters: {}.


Not using data augmentation.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_2 (Dropout)      

[I 2020-05-25 10:16:22,279] Finished trial#2 resulted in value: 0.9912999868392944. Current best value is 0.9927999973297119 with parameters: {}.


Study statistics: 
  Number of finished trials:  3
  Number of pruned trials:  0
  Number of complete trials:  3
Best trial:
  Value:  0.9927999973297119
  Params: 
